### **Imports**

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from robolab_turtlebot import Turtlebot
from TurtleControllers import TurtlebotController
from TurtleVision import TurtlebotVision
import TurtleUtils
import time

#### **Initialization**

In [2]:
turtle = Turtlebot(rgb = True, pc = True)
turtle_controller = TurtlebotController(turtle, rate = 40)
turtle_vision = TurtlebotVision(turtle)

#### **Find garage center**

In [ ]:
def find_garage_entrance(turtle_controller, turtle_vision):
    """
    Rotate turtlebot until it finds the center of the garage entrance.
    """
    PURPLE_MIN_SIZE = 15
    YELLOW_MIN_SIZE = 20

    def reg_mean(regs):
        s, n = 0.0, len(regs) * 2
        for reg in regs:
            s += reg[0] + reg[1]
        return s / n

    def reg_mid(reg):
        return (reg[0] + reg[1]) / 2

    # Find Garage center
    while True:

        # Find purple stripes
        regs_purple = turtle_vision.get_regions(color = "purple",
                                             minimal_size=PURPLE_MIN_SIZE)

        # Find yellow stripe inbetween purple stripes
        regs_yellow = turtle_vision.get_regions(color = "yellow",
                                                 minimal_size=YELLOW_MIN_SIZE)
        
        # Want to see two purple pillars and a yellow region inbetween
        while len(regs_purple) < 2 or len(regs_yellow) == 0:
            turtle_controller.cmd_velocity(angular = 0.5)
            regs_purple = turtle_vision.get_regions(color = "purple",
                                             minimal_size=PURPLE_MIN_SIZE)
            regs_yellow = turtle_vision.get_regions(color = "yellow",
                                                 minimal_size=YELLOW_MIN_SIZE)
            turtle_controller.rate.sleep()
            
        # Want to look at the center of the garage entrance
        purple_mid = reg_mean(regs_purple)
        err = purple_mid - turtle_vision.img_width // 2

        if abs(err) < 10:
            # Garage entrance found, stop!
            turtle_controller.cmd_velocity(linear = 0, angular = 0)
            break
        else:
            if err < 0.0:
                err = np.clip(err, -turtle_controller.max_yaw_rate, -turtle_controller.min_yaw_rate)
            else:
                err = np.clip(err, turtle_controller.min_yaw_rate, turtle_controller.max_yaw_rate)
            turtle_controller.cmd_velocity(linear = 0, angular = -err)
            
        turtle_controller.rate.sleep()

#### **Generate a garage model**

In [ ]:
def generate_path(turtle_vision, visualize = False):
    turtle_controller.reset_odometry()
    P1, P2, P_G, P_PG, P_GOAL = turtle_vision.garage_entry_waypoints()

    if visualize:
        fig, ax = plt.subplots(1)
        TurtleUtils.visualize_garage(ax, P1, P2, P_G, P_PG, P_GOAL)
    return P_PG, P_GOAL

#### **Find garage and enter it**

In [ ]:
def move_to_garage(turtle_controller, turtle_vision, pg_reset = False):
    # Find garage entrance
    find_garage_entrance()
    print("Garage found")
          
    P_PG, P_GOAL = generate_path(visualize = False)

    print("Moving to P_PG")
    turtle_controller.move_to(P_PG, relative = False)
    print("Done!")
    time.sleep(1)

    if pg_reset:
        find_garage_entrance()
        print("Garage found")
        P_PG, P_GOAL = generate_path(visualize = False)

    print("Moving to P_GOAL")
    turtle_controller.move_to(P_GOAL, relative = False)
    print("Done!")

## **Task1 solution**

In [ ]:
move_to_garage(turtle_controller, turtle_vision)